In [2]:
# ライブラリのインポート
import numpy as np
import pandas as pd
import psycopg2
import datetime

In [3]:
dbname = "kddi_18_param_custom2"
npyname = "trkn_18"
datedir = "230120"
sttime = "2022-04-01 22:00:00"

In [4]:
#解析結果の読み込み
timeseries_sep = np.load("./" + datedir+'/'+ npyname + ".npy")

In [5]:
timeseries_sep.shape

(24, 207)

In [6]:
timeseries_sep

array([[ 88,  62,  95, ..., 109, 109,   1],
       [ 77,  74,  90, ..., 116, 116,   0],
       [ 64,  74,  86, ..., 106, 106,   0],
       ...,
       [ 13,   4,  24, ...,  26,  26,   0],
       [ 21,  12,  12, ...,  28,  28,   0],
       [ 16,   6,  11, ...,  23,  23,   0]], dtype=int64)

In [7]:
#メッシュ情報の読み込み
csv_file = r'./'+datedir+'/'+ npyname+'_header.csv'
df = pd.read_csv(csv_file, dtype={'date':str}, index_col=0, parse_dates=True)
df_i = df.loc["gridcode"] #gridcode #end_mesh_area

In [14]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [ ]:
dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')

for lp in range(4):
    cur =connection.cursor()
    curs =connection.cursor()

    checkMeshtbranList = [] #確認対象メッシュのKDDI値

    #まずは時刻ごとのメッシュ一覧を取得
    sql = "select mesh from " + dbname + " WHERE times = '"+str(dte)+"'"
    cur.execute(sql)
    meshLists = cur.fetchall()
    
    print("time =" + str(dte) + " target =" + str(len(meshLists)))
    for mesh in meshLists:
        
        meshkey = str(int(mesh[0]))
        
        #トラカン値を持つメッシュならば解析しない
        find = 0
        for chkm in df_i.values:
            if str(int(chkm)) == meshkey:
                find = 1
                break
        if find == 1:
            continue

        sql = "select kddi_tran.tbran, T.gridcode,T.断面交通量,kddi_tran.mesh "
        sql += "from " + dbname + " AS kddi_tran "
        sql += "INNER JOIN pointdata as a ON kddi_tran.mesh = cast(a.grid_code AS BIGINT) "
        sql += "CROSS JOIN LATERAL "
        sql += "("
        sql += "SELECT 時刻, gridcode , 断面交通量 from trakan2204_z3out AS trakan_tran "
        sql += "inner JOIN pointdata as b ON trakan_tran.gridcode = cast(grid_code AS BIGINT) "
        sql += "WHERE 時刻 = '"+str(dte)+"' "
        sql += "ORDER BY a.geom <-> b.geom, ST_Distance(a.geom, b.geom) "
        sql += "LIMIT 1) AS T "
        sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + meshkey
        curs.execute(sql)
        datalists = curs.fetchall()
        #print(datalists) #(88.0, 50324526133, 13) #対象MESH値、確認対象メッシュ、対象の断面交通量
        
        if len(datalists) == 0:
            #print(sql)
            continue

        print(meshkey)
        #確認対象メッシュの kddi_tran.tbran値
        enabledcheck = meshkey in checkMeshtbranList
        baseMeshItem = 0
        if enabledcheck == True:
            for meshitem in checkMeshtbranList:
                if str(int(mesh)) == meshitem[0]:
                    baseMeshItem = meshitem[1]
                    break
        else:
            cur3 =connection.cursor()
            sql = "select kddi_tran.tbran from " + dbname + " AS kddi_tran "
            sql += "WHERE times = '"+str(dte)+"' and kddi_tran.mesh = " + str(datalists[0][1])
            cur3.execute(sql)
            baseMeshParam = cur3.fetchall()
            setparam = 0
            if len(baseMeshParam) != 0:
                setparam = baseMeshParam[0][0]
            if len(checkMeshtbranList) == 0:
                checkMeshtbranList = [meshkey , setparam]
            else:
                checkMeshtbranList.append([meshkey , setparam])
            cur3.close()
            baseMeshItem = setparam

        #メッシュ値を算出
        if baseMeshItem == 0:
            param = 0
        else:
            param = datalists[0][0] / baseMeshItem * datalists[0][2]
        print(str(datalists[0][0]) + ":" +  str(baseMeshItem) + ":" +  str( datalists[0][2]) )
            
        sql = "UPDATE " + dbname + " set tbran = "+ str(param) + " where times = '"  + str(dte) + "' and mesh = " + meshkey + ";"
        #print(sql)
        cur.execute(sql)
        connection.commit()
    cur.close()
    curs.close()
    dte = dte + datetime.timedelta(minutes=15)
connection.commit()

time =2022-04-01 22:00:00 target =6910
50325599211
0.002415885:23.0:62
50325599212
0.010056954:23.0:62
50325599213
0.026344012:23.0:62
50325599221
0.023330316:23.0:62
50325599222
0.014111672:65.0:11
50325599223
0.031647597:65.0:11
50325599224
0.016180912:65.0:11
50325599231
0.021973297:23.0:62
50325599232
0.0016901139:23.0:62
50325599233
0.015734654:23.0:62
50325599234
0.034116365:54.0:22
50325599242
0.009730853:65.0:11
50325599244
0.025338398:54.0:22
50325599311
0.02171536:114.0:18
50325599312
0.015335539:114.0:18
50325599313
0.0002528769:114.0:18
50325599314
0.048104532:114.0:18
50325599321
0.012364825:85.0:57
50325599322
0.015942015:85.0:57
50325599324
0.025366686:85.0:57
50325599331
0.0029549208:114.0:18
50325599332
0.029805548:85.0:57
50325599333
0.00021484471:114.0:18
50325599334
0.007953888:85.0:57
50325599342
0.03300947:85.0:57
50325599343
0.018841812:85.0:57
50325599412
0.007334741:54.0:22
50325599413
0.043688208:85.0:57
50325599414
0.0003478378:54.0:22
50325599421
0.06425085:

0.0049309046:88.0:21
50325650342
0.0014599767:88.0:21
50325650412
0.0065843277:88.0:21
50325650414
0.00025955355:88.0:21
50325650423
5.4916833e-05:88.0:21
50325651423
0.0019744134:57.0:16
50325651433
0.0065554297:4.0:12
50325651442
0.00071285525:57.0:16
50325652114
0.0018965644:86.0:40
50325652124
0.0068328413:86.0:40
50325652242
0.002337345:86.0:40
50325652243
0.0047749896:86.0:40
50325652244
0.0008650995:86.0:40
50325652313
0.0007364956:57.0:16
50325652314
0.010498972:57.0:16
50325652321
0.032693863:57.0:16
50325652422
0.04149673:86.0:40
50325652423
0.020092452:86.0:40
50325652424
0.03851649:86.0:40
50325652432
0.00038648513:86.0:40
50325652434
0.0021115746:86.0:40
50325653142
0.00033220788:48.0:8
50325653211
0.021826549:48.0:8
50325653212
0.0083191935:48.0:8
50325653214
0.04199071:48.0:8
50325653333
0.0002851151:86.0:40
50325653334
0.0023651365:86.0:40
50325653342
0.00228639:86.0:40
50325653343
0.038507573:86.0:40
50325653344
0.0010138112:86.0:40
50325653412
0.0010826854:86.0:40
503

0.011227084:38.0:40
50325673144
0.032505576:38.0:40
50325673211
0.097250044:38.0:40
50325673213
0.016479779:38.0:40
50325673313
0.0026790733:21.0:18
50325673314
0.007262865:21.0:18
50325673321
0.006952201:15.0:31
50325673322
0.0011348692:15.0:31
50325673323
0.016450882:15.0:31
50325673333
0.014112321:21.0:18
50325673434
0.004507916:89.0:21
50325673441
0.0067653228:89.0:21
50325673443
0.0050227:89.0:21
50325674111
0.0036877273:38.0:40
50325674112
0.006325393:38.0:40
50325674211
0.012824853:57.0:35
50325674212
0.055212714:57.0:35
50325674213
0.0004455559:57.0:35
50325674214
0.006092973:57.0:35
50325674222
0.0017282013:57.0:35
50325674324
0.03960351:28.0:25
50325674332
0.039990317:89.0:21
50325674333
1.014024e-05:89.0:21
50325674334
0.014902075:89.0:21
50325674341
0.02568391:89.0:21
50325674433
0.012478085:28.0:25
50325674441
0.0028235067:28.0:25
50325674443
0.048154615:28.0:25
50325675111
0.013912326:21.0:5
50325675212
0.018659536:21.0:5
50325675214
0.023913141:21.0:5
50325675224
0.00289

0.035927482:88.0:21
50325650312
0.00034829526:88.0:21
50325650313
0.00051406585:88.0:21
50325650314
0.0038714719:88.0:21
50325650321
0.00093293237:88.0:21
50325650322
0.015827583:88.0:21
50325650324
0.00089419633:88.0:21
50325650331
0.058536313:41.0:19
50325650332
0.010830624:88.0:21
50325651221
0.013731046:88.0:21
50325651222
0.0018792674:88.0:21
50325651242
0.0026171217:88.0:21
50325651331
0.00037232312:88.0:21
50325652222
0.051232558:44.0:24
50325652223
0.0035743164:44.0:24
50325652224
0.0019199103:44.0:24
50325652231
0.0007505386:86.0:40
50325652232
0.026221957:86.0:40
50325652234
0.014438154:86.0:40
50325652343
0.032986768:57.0:16
50325652344
0.0017165132:57.0:16
50325652412
0.0026995267:86.0:40
50325652413
0.023721343:86.0:40
50325652414
0.0027267986:86.0:40
50325653123
0.017899761:44.0:24
50325653124
0.025317613:48.0:8
50325653131
0.043648437:86.0:40
50325653132
0.01816905:44.0:24
50325653313
0.016388092:86.0:40
50325653314
0.01184378:86.0:40
50325653322
0.0009369054:86.0:40
503

50325672343
0.014814795:102.0:39
50325672344
0.0018132813:102.0:39
50325672412
0.075190276:102.0:39
50325672414
0.05901271:102.0:39
50325673122
0.04771661:38.0:40
50325673123
0.004700398:38.0:40
50325673124
0.002843853:38.0:40
50325673131
0.0017447933:38.0:38
50325673132
0.017563634:21.0:18
50325673133
0.013259161:21.0:18
50325673134
0.029840698:21.0:18
50325673233
0.031093683:38.0:40
50325673234
0.037351906:38.0:40
50325673241
0.0017019152:38.0:40
50325673243
0.006459959:38.0:40
50325673244
0.016947083:38.0:40
50325673312
0.01758891:21.0:18
50325673422
0.015894083:38.0:40
50325673423
0.051886648:38.0:40
50325673431
0.022950744:15.0:31
50325673433
0.0024965694:15.0:31
50325674132
0.019533155:38.0:40
50325674133
0.024446439:38.0:40
50325674134
0.047948442:38.0:40
50325674142
0.0111179305:57.0:35
50325674143
0.025131077:38.0:40
50325674144
0.014604062:28.0:25
50325674312
0.0018835869:38.0:40
50325674313
0.019513898:89.0:21
50325674314
0.05098226:89.0:21
50325674322
0.024534807:28.0:25
50

...

50326602142
0.03641262:61.0:9
50326602143
0.018620152:61.0:9
50326602144
0.0024302271:61.0:9
50326602211
0.02689369:111.0:17
50326602212
0.02084811:111.0:17
50326602213
0.020804722:111.0:17
50326602232
0.010000206:50.0:17
50326602234
0.0015639104:34.0:8
50326602241
0.025647461:50.0:17
50326602242
0.012328735:50.0:17
50326602243
0.010629381:50.0:17
50326602331
0.0062833135:188.0:41
50326602332
0.0044091376:35.0:9
50326602434
0.036845863:91.0:17
50326602441
0.013719294:91.0:17
50326602442
0.002785258:91.0:17
50326602444
0.0032497277:91.0:17
50326603111
0.019206276:50.0:17
50326603132
0.011038336:50.0:17
50326603133
0.043186344:50.0:17
50326603134
0.028550074:50.0:17
50326603142
0.003910895:50.0:17
50326603144
0.0025433907:34.0:8
50326603342
0.0065266537:34.0:8
50326603411
0.052537844:34.0:8
50326603412
0.04345506:62.0:16
50326603413
0.03868509:34.0:8
50326603421
0.0116320215:62.0:16
50326604121
0.025192317:27.0:12
50326604134
0.0046090707:62.0:16
50326604143
0.057403613:62.0:16
503266041

50326651112
0.032587387:121.0:47
50326651113
0.0034036897:121.0:47
50326651141
0.0023378166:134.0:36
50326652142
0.06046181:44.0:19
50326652144
0.0016571125:44.0:19
50326652211
0.0059013325:44.0:19
50326652213
0.00082090567:44.0:19
50326652341
0.0003317101:38.0:17
50326653121
0.00095938146:44.0:19
50326654133
0.0006648017:64.0:11
50326654211
0.0016340858:64.0:11
50326654212
0.014852769:64.0:11
50326654221
0.0073766215:64.0:11
50326654223
0.03192258:64.0:11
50326654231
0.00011861103:64.0:11
50326660132
0.0002027906:85.0:11
50326660133
0.022368254:85.0:11
50326660134
0.0046655256:85.0:11
50326660142
0.003020338:38.0:17
50326660143
0.007510196:38.0:17
50326660341
0.0031463765:38.0:17
50326660344
0.004431909:38.0:17
50326660413
0.0005963595:38.0:17
50326663142
0.0074056927:64.0:11
50326670113
0.0013937443:85.0:11
50326670114
0.0064591193:85.0:11
50326670131
0.0046910923:85.0:11
50326670134
0.015036328:61.0:22
50326670233
0.00036355155:61.0:22
50326670243
0.0012086323:61.0:22
50326670311
3.

0.00558431:62.0:16
50326623311
0.0075059235:41.0:8
50326623312
0.005945051:41.0:8
50326623323
0.0033091393:41.0:8
50326623331
0.023150377:41.0:8
50326623343
0.00012321863:41.0:8
50326624111
0.057921387:62.0:16
50326624144
0.0057914215:64.0:11
50326624211
0.0020513814:64.0:11
50326624212
0.0013683587:64.0:11
50326624213
0.00020763185:64.0:11
50326624214
0.015438834:64.0:11
50326624223
0.011625663:64.0:11
50326624231
0.004848508:64.0:11
50326624232
0.0028620968:64.0:11
50326624241
0.00038075913:64.0:11
50326624243
0.0010877472:64.0:11
50326624322
0.0010543312:64.0:11
50326624324
0.009133315:64.0:11
50326624334
0.040014334:64.0:11
50326630234
0.029240921:91.0:14
50326630242
0.0011917227:91.0:14
50326630243
0.014850048:91.0:14
50326630311
0.00232779:33.0:38
50326630312
0.002102193:52.0:16
50326630333
0.009296637:170.0:37
50326630334
0.027449707:170.0:37
50326630342
0.025289606:91.0:14
50326630344
0.018663583:170.0:37
50326630441
0.03681732:91.0:14
50326630442
0.0051973257:91.0:14
503266304

50327631431
0.050367557:11.0:2
50327631432
0.00077597983:11.0:2
50327632132
0.0009256994:11.0:2
50327632141
0.033261992:22.0:10
50327632142
0.00039166212:22.0:10
50327632144
0.019781295:22.0:10
50327632211
0.007995985:22.0:10
50327632344
0.00046162773:22.0:10
50327632411
0.009925888:22.0:10
50327632422
0.025412617:22.0:10
50327632423
0.0010785125:22.0:10
50327632424
0.010940498:22.0:10
50327632431
0.00037983956:22.0:10
50327632432
0.020617804:22.0:10
50327633312
0.05338433:22.0:10
50327633331
0.0019943553:22.0:10
50327633333
0.017248912:22.0:10
50327635343
0.0012194811:22.0:10
50327638133
0.024969272:22.0:10
50327641122
0.012716705:11.0:2
50327641124
0.0072191223:11.0:2
50327641211
0.001239553:11.0:2
50327641212
0.0012114802:11.0:2
50327641431
0.014881974:11.0:2
50327641441
0.0055894973:11.0:2
50327642121
0.012259455:22.0:10
50327642122
0.00908299:22.0:10
50327642212
0.0005839828:22.0:10
50327642213
0.0080993455:22.0:10
50327642214
0.0036539556:22.0:10
50327642221
0.0068484955:22.0:10


0.0057973894:99.0:26
50325506113
0.007991742:11.0:2
50325506114
0.04451476:99.0:26
50325506121
0.0093821185:99.0:26
50325506324
0.0013356077:99.0:26
50325506333
0.002200645:99.0:26
50325506342
0.02299833:99.0:26
50325507331
0.0029171915:99.0:26
50325507333
0.002400271:99.0:26
50325507411
0.00658457:99.0:26
50325507413
0.006137033:99.0:26
50325507414
0.003314231:99.0:26
50325507423
0.0023205117:99.0:26
50325509112
0.0064357407:99.0:26
50325509344
0.0054394305:99.0:26
50325515221
0.002245563:11.0:2
50325515222
0.0027046837:11.0:2
50325516114
0.0107253445:99.0:26
50325516121
0.04547532:99.0:26
50325516122
0.013028827:99.0:26
50325516123
0.022308987:99.0:26
50325516124
0.00085331476:99.0:26
50325516131
0.0073772753:61.0:3
50325516211
0.0036088016:99.0:26
50325516213
0.037630375:99.0:26
50325516221
0.003914959:99.0:26
50325516223
0.0013558611:99.0:26
50325516224
0.009473291:99.0:26
50325516231
0.011727838:61.0:3
50325516232
0.003305458:61.0:3
50325516332
0.0019065335:61.0:3
50325516333
0.00

In [ ]:
#ConvertMeshtoRoad

In [ ]:
connection.close()